<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

# Remove work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Failed to import module fastga.models.handling_qualities.handling_qualities.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.basicIC_engineOM.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_slipstream_x57.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
api_cs23.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

INFO    : Sample configuration written in workdir\oad_process.yml


You can now checkout the generated [configuration file](./workdir/oad_process.toml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 149 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cl_extreme (ID 28 )
INFO    : Installed bundle fastga.models.loops.unitary_tests (ID 416 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 152 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion (ID 433 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cnbeta_fuselage (ID 29 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.c_systems.c2_life_support_systems_weight (ID 187 )
INFO    : Installed bundle fastga.utils.postprocessing.analysis_and_plots_vn (ID 211 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a3_tail_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.nacelle

INFO    : Installed bundle fastga.models.load_analysis.aerodynamic_loads (ID 115 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c2_life_support_systems_cg (ID 159 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 431 )
INFO    : Installed bundle fastga.notebooks.tutorial.img (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c3_navigation_systems_cg (ID 160 )
INFO    : Installed bundle fastga.models.performances.unitary_tests (ID 418 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 145 )
INFO    : Installed bundle fastga.notebooks.tutorial.data (ID 439 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_reynolds (ID 33 )
INFO    : Installed bundle fastga.utils.physics (ID 442 )
INFO    : Installed bundle fastga.models.aerodynamics.components.high_lift_aero (ID 34 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_mfw (ID 

INFO    : Installed bundle fastga.models (ID 384 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.xfoil_group (ID 54 )
INFO    : Installed bundle fastga.models.geometry.geom_components.compute_total_area (ID 64 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_tail_areas (ID 109 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.compute_vn (ID 43 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 394 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_ht_area (ID 108 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 118 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 178 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion.b2_fuel_lines_weight (ID 183 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing (ID 411 )


'D:\\fl.lutz\\FAST\\FAST-OAD\\FAST-GA\\src\\fastga\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [5]:
api_cs25.list_variables(CONFIGURATION_FILE)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 149 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cl_extreme (ID 28 )
INFO    : Installed bundle fastga.models.loops.unitary_tests (ID 416 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 152 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion (ID 433 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cnbeta_fuselage (ID 29 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.c_systems.c2_life_support_systems_weight (ID 187 )
INFO    : Installed bundle fastga.utils.postprocessing.analysis_and_plots_vn (ID 211 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a3_tail_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.nacelle

INFO    : Installed bundle fastga.models.load_analysis.aerodynamic_loads (ID 115 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c2_life_support_systems_cg (ID 159 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 431 )
INFO    : Installed bundle fastga.notebooks.tutorial.img (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c3_navigation_systems_cg (ID 160 )
INFO    : Installed bundle fastga.models.performances.unitary_tests (ID 418 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 145 )
INFO    : Installed bundle fastga.notebooks.tutorial.data (ID 439 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_reynolds (ID 33 )
INFO    : Installed bundle fastga.utils.physics (ID 442 )
INFO    : Installed bundle fastga.models.aerodynamics.components.high_lift_aero (ID 34 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_mfw (ID 

INFO    : Installed bundle fastga.models (ID 384 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.xfoil_group (ID 54 )
INFO    : Installed bundle fastga.models.geometry.geom_components.compute_total_area (ID 64 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_tail_areas (ID 109 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.compute_vn (ID 43 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 394 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_ht_area (ID 108 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 118 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 178 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion.b2_fuel_lines_weight (ID 183 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing (ID 411 )


NAME,I/O,DESCRIPTION
data:TLAR:NPAX_design,IN,
data:TLAR:luggage_mass_design,IN,
data:TLAR:range,IN,top-level requirement: design range
data:TLAR:v_approach,IN,
data:TLAR:v_cruise,IN,
data:TLAR:v_limit,IN,
data:TLAR:v_max_sl,IN,
data:geometry:cabin:aisle_width,IN,width of aisles
data:geometry:cabin:luggage:mass_max,IN,
data:geometry:cabin:seats:passenger:NPAX_max,IN,


Another useful feature is to list the modules of the model defined in the configuration file:

In [6]:
api_cs25.list_modules(CONFIGURATION_FILE)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 149 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cl_extreme (ID 28 )
INFO    : Installed bundle fastga.models.loops.unitary_tests (ID 416 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 152 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion (ID 433 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cnbeta_fuselage (ID 29 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.c_systems.c2_life_support_systems_weight (ID 187 )
INFO    : Installed bundle fastga.utils.postprocessing.analysis_and_plots_vn (ID 211 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a3_tail_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.nacelle

INFO    : Installed bundle fastga.models.load_analysis.aerodynamic_loads (ID 115 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c2_life_support_systems_cg (ID 159 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 431 )
INFO    : Installed bundle fastga.notebooks.tutorial.img (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c3_navigation_systems_cg (ID 160 )
INFO    : Installed bundle fastga.models.performances.unitary_tests (ID 418 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 145 )
INFO    : Installed bundle fastga.notebooks.tutorial.data (ID 439 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_reynolds (ID 33 )
INFO    : Installed bundle fastga.utils.physics (ID 442 )
INFO    : Installed bundle fastga.models.aerodynamics.components.high_lift_aero (ID 34 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_mfw (ID 

INFO    : Installed bundle fastga.models (ID 384 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.xfoil_group (ID 54 )
INFO    : Installed bundle fastga.models.geometry.geom_components.compute_total_area (ID 64 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_tail_areas (ID 109 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.compute_vn (ID 43 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 394 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_ht_area (ID 108 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 118 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 178 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion.b2_fuel_lines_weight (ID 183 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing (ID 411 )


AVAILABLE MODULE IDENTIFIERS,MODULE PATH
fastga.aerodynamics.highspeed.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_high_speed.py
fastga.aerodynamics.lowspeed.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_low_speed.py
fastga.geometry.alternate,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.geometry.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.handling_qualities.static_margin,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\compute_static_margin.py
fastga.handling_qualities.tail_sizing,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\tail_sizing\update_tail_areas.py
fastga.loads.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\load_analysis\loads.py
fastga.loop.mtow,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\weight\mass_breakdown\update_mtow.py
fastga.loop.wing_area,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\loops\update_wing_area.py
fastga.loop.wing_position,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\loops\update_wing_position.py


Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [7]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\n2.html


Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [ ]:
XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
api_cs25.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=XDSM_FILE, width='100%', height='500px')

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 175 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 186 )
INFO    : Installed bundle fastga.models.geometry.profiles (ID 417 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 122 )
INFO    : Installed bundle fastga.models.performances.mission (ID 137 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine (ID 430 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 403 )
INFO    : Installed bundle fastga.models.performances.takeoff (ID 138 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.update_mlg (ID 176 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a3_tail_weigh

INFO    : Installed bundle fastga.models.weight.cg.cg_components.b_propulsion (ID 436 )
INFO    : Installed bundle fastga.models.geometry.geom_components.fuselage (ID 409 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engine_be76 (ID 202 )
INFO    : Installed bundle fastga.models.aerodynamics.unitary_tests.dummy_engine_x57 (ID 62 )
INFO    : Installed bundle fastga.models.handling_qualities (ID 419 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.openmdao (ID 147 )
INFO    : Installed bundle fastga.models.weight.cg.unitary_tests.dummy_engine_sr22 (ID 180 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems (ID 437 )
INFO    : Installed bundle fastga.models.load_analysis.structural_loads_x57 (ID 126 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests.test_basicIC_engine (ID 148 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.

INFO    : Installed bundle fastga.models.geometry.geom_components.ht.compute_horizontal_tail (ID 76 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_wing_mac (ID 91 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_wing_sweep (ID 92 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests.dummy_engine_be76 (ID 115 )
INFO    : Installed bundle fastga.models.weight (ID 432 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.b_propulsion.b1_engine_cg (ID 161 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_wing_toc (ID 93 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_wing_wet_area (ID 94 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundl

You have to set your API key.
You can get it in your profile page on WhatsOpt (https://selene.onecert.fr/whatsopt).
Please, copy/paste your API key below then hit return (characters are hidden).


## 2. Running your first MDA

### Beechcraft
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [8]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 149 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cl_extreme (ID 28 )
INFO    : Installed bundle fastga.models.loops.unitary_tests (ID 416 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 152 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion (ID 433 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cnbeta_fuselage (ID 29 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.c_systems.c2_life_support_systems_weight (ID 187 )
INFO    : Installed bundle fastga.utils.postprocessing.analysis_and_plots_vn (ID 211 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a3_tail_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.nacelle

INFO    : Installed bundle fastga.models.load_analysis.aerodynamic_loads (ID 115 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c2_life_support_systems_cg (ID 159 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 431 )
INFO    : Installed bundle fastga.notebooks.tutorial.img (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c3_navigation_systems_cg (ID 160 )
INFO    : Installed bundle fastga.models.performances.unitary_tests (ID 418 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 145 )
INFO    : Installed bundle fastga.notebooks.tutorial.data (ID 439 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_reynolds (ID 33 )
INFO    : Installed bundle fastga.utils.physics (ID 442 )
INFO    : Installed bundle fastga.models.aerodynamics.components.high_lift_aero (ID 34 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_mfw (ID 

INFO    : Installed bundle fastga.models (ID 384 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.xfoil_group (ID 54 )
INFO    : Installed bundle fastga.models.geometry.geom_components.compute_total_area (ID 64 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_tail_areas (ID 109 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.compute_vn (ID 43 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 394 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_ht_area (ID 108 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 118 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 178 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion.b2_fuel_lines_weight (ID 183 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing (ID 411 )


NL: NLBGS 1 ; 12257370.8 1
NL: NLBGS 2 ; 4603590.62 0.375577332
NL: NLBGS 3 ; 4662716.16 0.380401003
NL: NLBGS 4 ; 2763691.85 0.225471832
NL: NLBGS 5 ; 757195.255 0.0617746878
NL: NLBGS 6 ; 280399.389 0.0228759816
NL: NLBGS 7 ; 98135.9576 0.00800628121
NL: NLBGS 8 ; 66151.3341 0.00539686162
NL: NLBGS 9 ; 66209.4795 0.00540160533
NL: NLBGS 10 ; 35216.7995 0.00287311203
NL: NLBGS 11 ; 20284.1478 0.00165485307
NL: NLBGS 12 ; 13435.5675 0.00109612148
NL: NLBGS 13 ; 8604.45242 0.000701981897
NL: NLBGS 14 ; 4858.47199 0.000396371462
NL: NLBGS 15 ; 2859.02966 0.00023324983
NL: NLBGS 16 ; 1682.32031 0.000137249687
NL: NLBGS 17 ; 963.432859 7.86002867e-05
NL: NLBGS 18 ; 538.082892 4.38987202e-05


D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\external\xfoil\xfoil_polar.py:418: UserWarning: genfromtxt: Empty input file: "C:\Users\FL36F6~1.LUT\AppData\Local\Temp\xcg45aooj\out"
  result_array = np.genfromtxt(xfoil_result_file_path, skip_header=12, dtype=dtypes)


NL: NLBGS 19 ; 329.909192 2.69151678e-05
NL: NLBGS 20 ; 214.677614 1.75141649e-05
NL: NLBGS 21 ; 90.2796616 7.36533658e-06
NL: NLBGS Converged


INFO    : Computation finished after 300.16 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [9]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
Beechcraft_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mda.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [10]:
api_cs25.variable_viewer(OUTPUT_FILE)

In particular, you may inspect the `data:handling_qualities:static_margin` variable.
You will see that its value is slightly negative, which means that the current aircraft configuration is estimated as unstable. We will see later how we can correct this point.

### Beechcraft for 1000 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 1000 nm range instead of 800 nm for the Beechcraft. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [11]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
api_cs25.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

In [12]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 149 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cl_extreme (ID 28 )
INFO    : Installed bundle fastga.models.loops.unitary_tests (ID 416 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 152 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion (ID 433 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cnbeta_fuselage (ID 29 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.c_systems.c2_life_support_systems_weight (ID 187 )
INFO    : Installed bundle fastga.utils.postprocessing.analysis_and_plots_vn (ID 211 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a3_tail_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.nacelle

INFO    : Installed bundle fastga.models.load_analysis.aerodynamic_loads (ID 115 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c2_life_support_systems_cg (ID 159 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 431 )
INFO    : Installed bundle fastga.notebooks.tutorial.img (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c3_navigation_systems_cg (ID 160 )
INFO    : Installed bundle fastga.models.performances.unitary_tests (ID 418 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 145 )
INFO    : Installed bundle fastga.notebooks.tutorial.data (ID 439 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_reynolds (ID 33 )
INFO    : Installed bundle fastga.utils.physics (ID 442 )
INFO    : Installed bundle fastga.models.aerodynamics.components.high_lift_aero (ID 34 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_mfw (ID 

INFO    : Installed bundle fastga.models (ID 384 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.xfoil_group (ID 54 )
INFO    : Installed bundle fastga.models.geometry.geom_components.compute_total_area (ID 64 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_tail_areas (ID 109 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.compute_vn (ID 43 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 394 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_ht_area (ID 108 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 118 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 178 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion.b2_fuel_lines_weight (ID 183 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing (ID 411 )


NL: NLBGS 1 ; 12304844.4 1
NL: NLBGS 2 ; 4603590.8 0.374128323
NL: NLBGS 3 ; 4698544.96 0.381845134
NL: NLBGS 4 ; 2789979.08 0.226738265
NL: NLBGS 5 ; 866580.365 0.0704259506
NL: NLBGS 6 ; 309142.564 0.0251236468
NL: NLBGS 7 ; 118519.925 0.00963197263
NL: NLBGS 8 ; 77077.1675 0.00626396929
NL: NLBGS 9 ; 82067.3913 0.00666951882
NL: NLBGS 10 ; 43664.1878 0.00354853636
NL: NLBGS 11 ; 25332.8115 0.00205876731
NL: NLBGS 12 ; 16536.3752 0.00134389145
NL: NLBGS 13 ; 11079.068 0.000900382613
NL: NLBGS 14 ; 6471.08014 0.00052589695
NL: NLBGS 15 ; 3848.93117 0.000312798037
NL: NLBGS 16 ; 2321.78906 0.000188689022
NL: NLBGS 17 ; 1384.31999 0.000112502032
NL: NLBGS 18 ; 796.713111 6.47479224e-05
NL: NLBGS 19 ; 456.211885 3.70757946e-05
NL: NLBGS 20 ; 264.997836 2.15360574e-05
NL: NLBGS 21 ; 151.04151 1.2274963e-05
NL: NLBGS 22 ; 85.2112815 6.92501902e-06
NL: NLBGS Converged


INFO    : Computation finished after 158.96 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save again these new results, for post-processing them in next notebook.

In [13]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
Beechcraft_1000nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_1000nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_1000nm_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_1000nm_mda.xml'

## 3. Running your first MDO

## Beechcraft 800 nm optimization

As seen earlier, the current aircraft configuration is estimated with a fixed rear length (and thus fuselage length). We will unlock this parameter using a simple optimization problem that will move horizontal tail position in order to minimize the mission consumption. 

The default configuration file defines this optimization problem that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the rear fusellage length (design variables): by extension the horizontal tail to wing distance,
- subject to no additional constraint.

*(This run should take approximately 20 minutes)*

In [14]:
# Set back the inputs from the reference Beechcraft 800 nm
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 149 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cl_extreme (ID 28 )
INFO    : Installed bundle fastga.models.loops.unitary_tests (ID 416 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 152 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion (ID 433 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cnbeta_fuselage (ID 29 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.c_systems.c2_life_support_systems_weight (ID 187 )
INFO    : Installed bundle fastga.utils.postprocessing.analysis_and_plots_vn (ID 211 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a3_tail_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.nacelle

INFO    : Installed bundle fastga.models.load_analysis.aerodynamic_loads (ID 115 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c2_life_support_systems_cg (ID 159 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 431 )
INFO    : Installed bundle fastga.notebooks.tutorial.img (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c3_navigation_systems_cg (ID 160 )
INFO    : Installed bundle fastga.models.performances.unitary_tests (ID 418 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 145 )
INFO    : Installed bundle fastga.notebooks.tutorial.data (ID 439 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_reynolds (ID 33 )
INFO    : Installed bundle fastga.utils.physics (ID 442 )
INFO    : Installed bundle fastga.models.aerodynamics.components.high_lift_aero (ID 34 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_mfw (ID 

INFO    : Installed bundle fastga.models (ID 384 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.xfoil_group (ID 54 )
INFO    : Installed bundle fastga.models.geometry.geom_components.compute_total_area (ID 64 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_tail_areas (ID 109 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.compute_vn (ID 43 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 394 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_ht_area (ID 108 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 118 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 178 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion.b2_fuel_lines_weight (ID 183 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing (ID 411 )


'D:\\fl.lutz\\FAST\\FAST-OAD\\FAST-GA\\src\\fastga\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [15]:
api_cs25.optimization_viewer(CONFIGURATION_FILE)

In [16]:
optim_problem = api_cs25.optimize_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 149 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cl_extreme (ID 28 )
INFO    : Installed bundle fastga.models.loops.unitary_tests (ID 416 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 152 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion (ID 433 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_cnbeta_fuselage (ID 29 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.c_systems.c2_life_support_systems_weight (ID 187 )
INFO    : Installed bundle fastga.utils.postprocessing.analysis_and_plots_vn (ID 211 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a3_tail_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.nacelle

INFO    : Installed bundle fastga.models.load_analysis.aerodynamic_loads (ID 115 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c2_life_support_systems_cg (ID 159 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown (ID 431 )
INFO    : Installed bundle fastga.notebooks.tutorial.img (ID 440 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.c_systems.c3_navigation_systems_cg (ID 160 )
INFO    : Installed bundle fastga.models.performances.unitary_tests (ID 418 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 145 )
INFO    : Installed bundle fastga.notebooks.tutorial.data (ID 439 )
INFO    : Installed bundle fastga.models.aerodynamics.components.compute_reynolds (ID 33 )
INFO    : Installed bundle fastga.utils.physics (ID 442 )
INFO    : Installed bundle fastga.models.aerodynamics.components.high_lift_aero (ID 34 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components.compute_mfw (ID 

INFO    : Installed bundle fastga.models (ID 384 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.xfoil_group (ID 54 )
INFO    : Installed bundle fastga.models.geometry.geom_components.compute_total_area (ID 64 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_tail_areas (ID 109 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.compute_vn (ID 43 )
INFO    : Installed bundle fastga.models.aerodynamics.external.xfoil.resources (ID 394 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.update_ht_area (ID 108 )
INFO    : Installed bundle fastga.models.load_analysis.aerostructural_loads_x57 (ID 118 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.a_airframe.a4_flight_control_weight (ID 178 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.b_propulsion.b2_fuel_lines_weight (ID 183 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing (ID 411 )


NL: NLBGS 1 ; 12257370.8 1
NL: NLBGS 2 ; 4603590.62 0.375577332
NL: NLBGS 3 ; 4662716.16 0.380401003
NL: NLBGS 4 ; 2763691.85 0.225471832
NL: NLBGS 5 ; 757195.255 0.0617746878
NL: NLBGS 6 ; 280399.389 0.0228759816
NL: NLBGS 7 ; 98135.9576 0.00800628121
NL: NLBGS 8 ; 66151.3341 0.00539686162
NL: NLBGS 9 ; 66209.4795 0.00540160533
NL: NLBGS 10 ; 35216.7995 0.00287311203
NL: NLBGS 11 ; 20284.1478 0.00165485307
NL: NLBGS 12 ; 13435.5675 0.00109612148
NL: NLBGS 13 ; 8604.45242 0.000701981897
NL: NLBGS 14 ; 4858.47199 0.000396371462
NL: NLBGS 15 ; 2859.02966 0.00023324983
NL: NLBGS 16 ; 1682.32031 0.000137249687
NL: NLBGS 17 ; 963.432859 7.86002867e-05
NL: NLBGS 18 ; 538.082892 4.38987202e-05
NL: NLBGS 19 ; 303.849104 2.4789093e-05
NL: NLBGS 20 ; 171.961279 1.40292141e-05
NL: NLBGS 21 ; 90.2796616 7.36533658e-06
NL: NLBGS Converged
NL: NLBGS 1 ; 82.8886678 1
NL: NLBGS 2 ; 25.9429424 0.312985395
NL: NLBGS 3 ; 13.5066868 0.162949739
NL: NLBGS 4 ; 7.07587185 0.0853659739
NL: NLBGS 5 ; 3.6625372

NL: NLBGS 25 ; 0.00748285129 5.65497675e-05
NL: NLBGS 26 ; 0.00340074499 2.57002754e-05
NL: NLBGS 27 ; 0.00152659785 1.15368795e-05
NL: NLBGS 28 ; 0.000678762484 5.12957684e-06
NL: NLBGS Converged
NL: NLBGS 1 ; 65.8065622 1
NL: NLBGS 2 ; 59882.7052 909.980755
NL: NLBGS 3 ; 11049.6003 167.91031
NL: NLBGS 4 ; 9370.6972 142.39761
NL: NLBGS 5 ; 1714.64516 26.0558386
NL: NLBGS 6 ; 596.105855 9.0584561
NL: NLBGS 7 ; 666.010823 10.1207357
NL: NLBGS 8 ; 210.060832 3.19209551
NL: NLBGS 9 ; 84.8937489 1.2900499
NL: NLBGS 10 ; 77.7218713 1.18106567
NL: NLBGS 11 ; 43.1675567 0.655976474
NL: NLBGS 12 ; 20.6408866 0.313660004
NL: NLBGS 13 ; 12.0590726 0.183250305
NL: NLBGS 14 ; 6.94355669 0.105514655
NL: NLBGS 15 ; 3.58511258 0.0544795604
NL: NLBGS 16 ; 1.96359916 0.0298389566
NL: NLBGS 17 ; 0.995145712 0.0151222869
NL: NLBGS 18 ; 0.571711906 0.00868776435
NL: NLBGS 19 ; 0.285250773 0.00433468584
NL: NLBGS 20 ; 0.246727166 0.00374927907
NL: NLBGS 21 ; 0.14187751 0.00215597814
NL: NLBGS 22 ; 0.045790

NL: NLBGS 93 ; 0.365439215 0.0144404288
NL: NLBGS 94 ; 0.365439215 0.0144404288
NL: NLBGS 95 ; 0.365439215 0.0144404288
NL: NLBGS 96 ; 0.365439215 0.0144404288
NL: NLBGS 97 ; 0.365439215 0.0144404288
NL: NLBGS 98 ; 0.365439215 0.0144404288
NL: NLBGS 99 ; 0.365439215 0.0144404288
NL: NLBGS 100 ; 0.365439215 0.0144404288
NL: NLBGSSolver 'NL: NLBGS' on system '' failed to converge in 100 iterations.
NL: NLBGS 1 ; 13.8023106 1
NL: NLBGS 2 ; 11321.0229 820.226641
NL: NLBGS 3 ; 2283.68742 165.456892
NL: NLBGS 4 ; 1841.85389 133.44533
NL: NLBGS 5 ; 317.43538 22.998713
NL: NLBGS 6 ; 124.108365 8.99185424
NL: NLBGS 7 ; 135.818389 9.84026467
NL: NLBGS 8 ; 51.7325519 3.74810809
NL: NLBGS 9 ; 21.2444433 1.5391947
NL: NLBGS 10 ; 17.8594351 1.29394531
NL: NLBGS 11 ; 9.42701044 0.683002341
NL: NLBGS 12 ; 4.8278396 0.349784884
NL: NLBGS 13 ; 2.66094565 0.192789869
NL: NLBGS 14 ; 1.54342327 0.111823543
NL: NLBGS 15 ; 0.744863863 0.0539666064
NL: NLBGS 16 ; 0.501995155 0.0363703709
NL: NLBGS 17 ; 0.1882

INFO    : Computation finished after 1125.54 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results:

In [17]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mdo.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mdo.xml'

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [18]:
api_cs25.optimization_viewer(CONFIGURATION_FILE)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [19]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
api_cs25.variable_viewer(RESULT_FILE)